This notebook demonstrates how to load the V1 SLR prediction model (using ERA5 data) from Veals et al (2025), and apply it to a sample dataframe of atmospheric variables to yield a predicted SLR. It then generates a comparison plot between the observed SLR and model-predicted SLR.

In [ ]:
import numpy as np
import pandas as pd

import pickle
from matplotlib import pyplot as plt
import time

from matplotlib.colors import BoundaryNorm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score   
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

import matplotlib.gridspec as gridspec
from copy import copy
pd.set_option('display.max_columns', None)

In [ ]:
slr_model_path = '~/Veals_etal_2025/Models/'
slr_data_path = '~/Veals_etal_2025/Data/'

In [ ]:
#Load the atmospheric data from the ERA5, and the observed SLRs. 
AtmData = pd.read_pickle(slr_data_path+'V1a_atmospheric_data.pd')
ObsSLR = pd.read_pickle(slr_data_path+'V1a_observed_slr.pd')

#Then load the trained Multiple Linear Regression model and the model keys.
model = np.load(slr_model_path+'V1a_slr_model.npy', allow_pickle=True)[()]
keys = np.load(slr_model_path+'V1a_slr_model_keys.npy', allow_pickle=True)

In [ ]:
#The keys will tell you which variables the Linear Regression expects to be fed into it
AtmData = pd.DataFrame(AtmData.loc[:, list(keys)])
Predicted_SLR = model.predict(AtmData)
display(Predicted_SLR)

In [ ]:
#Plot Predicted SLR vs Observed SLR
plt.rcParams.update({'font.size': 18})


fig = plt.figure(figsize=(10,7))
gs = gridspec.GridSpec(1, 1, hspace=0.15, wspace=0.05, left=0.09, bottom=0.09, right=0.96, top=0.96);
ax=plt.subplot(gs[0, 0]); 

bins = np.arange(0, 51, 1.0)
heatmap, xedges, yedges = np.histogram2d(ObsSLR, Predicted_SLR, bins = bins)
bin_counts = np.sum(heatmap)
bin_percents = (heatmap / bin_counts)
cmap = copy(plt.get_cmap('plasma'))
cmap.set_under('white', 1)
    
X, Y = np.meshgrid(xedges, yedges)
    
levels = [0, 1, 2, 4, 6, 8, 10, 12, 14, 16]
levels[0] = 1e-5
norm = BoundaryNorm(levels, ncolors = cmap.N, clip = True)

heatmap = np.where(heatmap > 0, heatmap, np.nan)
cbd = ax.pcolormesh(heatmap, cmap = cmap, norm = norm)

ax.set_xticks(np.arange(0, 46, 5))
ax.set_yticks(np.arange(0, 46, 5))
ax.axline((0, 0), (1, 1), linewidth = 0.5, color = 'k', zorder = 1)
ax.grid(True, which = 'major', linestyle = '-', color = 'k', alpha = 0.5)
ax.grid(True, which = 'minor', color = 'k', linestyle = '-', alpha = 0.5)
ax.tick_params(axis = 'both', which = 'major', labelsize = 16, pad = 5)
ax.set_aspect('equal', adjustable = 'box')
ax.set_ylabel('Observed SLR',fontsize=24,fontweight='bold')
ax.set_xlabel('Predicted SLR',fontsize=24,fontweight='bold')
CB=plt.colorbar(cbd)
CB.set_label('n events',fontsize=20,fontweight='bold')

ax.set_yticklabels(np.arange(0, 46, 5))
ax.set_xticklabels(np.arange(0, 46, 5))
ax.set_xlim(0, 50)
ax.set_ylim(0, 50)


fig.subplots_adjust(top=.85)
plt.show()